In [46]:
import pandas as pd
import folium
import json
import numpy as np
from IPython.display import HTML

In [47]:
shape_url = 'https://raw.githubusercontent.com/Ccantey/LCC-DistrictFinder/master/data/HSE2012.json'

#District_geo1 = "L2012-1.json"
District_geo = shape_url
#District_geo = json.dumps(json.load(open(District_geo1, 'r')))

def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))

def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

def patcher(fn='map.html'):
    f=open(fn,'r')
    html=f.read()
    f.close()
    html=html.replace('"//','"http://')
    f=open(fn,'w')
    f.write(html)
    f.close()

from IPython.display import HTML

def display(m, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: {1}px; '
                 'border: none"></iframe>'.format(srcdoc, height))
    return embed

#http://nbviewer.ipython.org/gist/psychemedia/fbcd7cf1daabe0004e27/folium_shapefiles.ipynb 
#Examples - http://blog.ouseful.info/2015/04/17/creating-interactive-election-maps-using-folium-and-ipython-notebooks/ 





In [56]:
data = pd.read_csv('stateHouseStats2016.csv')
data['DFL_Percentage'] = data['MNLEGPERC2014'] * 100
data['Dayton_Percentage'] = data['MNGOVPERC2014'] * 100
data['Pickup_code'] = data['Pickup_code'] * 40
data.head()

,District,DPI,DemBase,MidTermDropOffDemBase,Name,Party,Pickup,Pickup_code,YearElected,MNLEGPERC2014,MNGOVPERC2014,District2010,DFL_Percentage,Dayton_Percentage
0,10A,0.426354,0.384084,0.378168,Josh Heintzeman,Republican,Republican,0,2014,0.465370,0.414684,10A,46.537037,41.468376
1,10B,0.436126,0.399135,0.380867,Dale Lueck,Republican,Republican,0,2014,0.479980,0.454321,10B,47.997973,45.432099
2,11A,0.529618,0.496667,0.453965,Mike Sundin,DFL,Incumbent,40,2012,0.623473,0.592162,11A,62.347301,59.216246
3,11B,0.447436,0.409999,0.366094,Jason Rarick,Republican,Republican,0,2014,0.461517,0.459324,11B,46.151657,45.932371
4,12A,0.463710,0.401893,0.209892,Jeff Backer,Republican,Republican,0,2014,0.479261,0.472669,12A,47.926076,47.266900


In [57]:
MN_COORDS = [44.9543070,-93.1022220]
stateHouse_map = folium.Map(location = MN_COORDS, zoom_start = 10)

stateHouse_map.geo_json(geo_path = District_geo, 
              data_out="data1.json", 
              data=data, 
              columns=['Name','DFL_Percentage'], 
              key_on='feature.properties.name',
              threshold_scale=[ 40, 45, 50, 55, 60],
              fill_color = 'RdBu',
              fill_opacity=0.4, 
              line_opacity=0.9,
              legend_name = 'Vote Percentage',
              reset="True")
embed_map(stateHouse_map)


In [58]:
gov_map = folium.Map(location = MN_COORDS, zoom_start = 10)

gov_map.geo_json(geo_path = District_geo, 
              data_out="data2.json", 
              data=data, 
              columns=['Name','Dayton_Percentage'], 
              key_on='feature.properties.name',
              #threshold_scale=[0, 40, 50, 65],
              fill_color = 'RdBu',
              fill_opacity=0.4, 
              line_opacity=0.9,
              legend_name = 'Gubernatorial Percentage',
              reset="True")
embed_map(gov_map)


In [63]:
flip_map = folium.Map(location = MN_COORDS, zoom_start = 10)

flip_map.geo_json(geo_path = District_geo, 
              data_out="data2.json", 
              data=data, 
              columns=['Name','Pickup_code'], 
              key_on='feature.properties.name',
              threshold_scale=[20, 40, 50, 80],
              fill_color = 'RdBu',
              fill_opacity=0.4, 
              line_opacity=0.9,
              legend_name = 'Pickup',
              reset="True")
embed_map(flip_map)

# Everything below here is code I have not gotten to work yet.  Goal was to color in choropleth by whehter it was a newly elected member or incumbent.

Got it working above.


In [52]:
'''def pickup_color_function(feature):
    if elect_dict[feature['Pickup']] == "Republican":
        return '#FF0000'
    elif elect_dict[feature['Pickup']] == "DFL":
        return '#0000FF'
    else:
        return '#1E1E1E'
        '''

'def pickup_color_function(feature):\n    if elect_dict[feature[\'Pickup\']] == "Republican":\n        return \'#FF0000\'\n    elif elect_dict[feature[\'Pickup\']] == "DFL":\n        return \'#0000FF\'\n    else:\n        return \'#1E1E1E\'\n        '

In [53]:
'''elect_dict = data.set_index('Name')['Pickup']
elect_dict'''

"elect_dict = data.set_index('Name')['Pickup']\nelect_dict"

In [54]:
'''m = folium.Map(location = MN_COORDS, zoom_start = 10)

m.geo_json(
    District_geo,
    fillColor= pickup_color_function(elect_dict)

    )
embed_map(m)'''

'm = folium.Map(location = MN_COORDS, zoom_start = 10)\n\nm.geo_json(\n    District_geo,\n    fillColor= pickup_color_function(elect_dict)\n\n    )\nembed_map(m)'